In [5]:
import sys
import time

import ollama
import jericho
import re
import json
import random

In [4]:
response = ollama.generate(model='llama3.2:1b', prompt='What is a large language model?')
print(response)

KeyboardInterrupt: 

In [6]:
GAMES_DIR = "z-machine-games-master/jericho-game-suite"
game = 'zork1.z5'
env = jericho.FrotzEnv(f"{GAMES_DIR}/{game}")

# System Prompts

In [ ]:
# Basic
system_prompt = (
    'You are a smart video game enthusiast who is skilled at playing old-school text adventure games. Given a description of your environment, explain your thought process and then give a command that is compatible with the game you are playing. Always put the command on its own line at the end of your response with nothing else. It needs to be easy and consistent to read with simple python'
    'This game does not run on an LLM, so it only recognizes a small vocabulary of commands. An example of a command is exactly the string "go west". This means you should only give exactly the command that the game recognizes '
)

In [ ]:
def basic_llm(max_retry=5, out_file=sys.stdout):
    original_stdout = sys.stdout
    sys.stdout = out_file

    make_prompt = lambda x: f'{system_prompt}\n\n{x}'

    print("Playing game. 'q' to quit")

    initial_observation, info = env.reset()
    print(initial_observation)

    game_response = initial_observation

    done = False
    total_steps = 0
    retries = 0
    retries_per_score = []
    generate_times = []
    prev_score = -1
    while not done and retries <= max_retry:

        prompt = make_prompt(f'Game prompt:\n{game_response}')
        start = time.time()
        response = ollama.generate(model='llama3.2:1b', prompt=prompt).response
        generate_times.append(time.time() - start)
        print(f'[r{retries}] LLM Response: {response}')
        player_in = response.splitlines()[-1].strip()
        print(player_in)

        # Take an action in the environment using the step fuction.
        # The resulting text-observation, reward, and game-over indicator is returned.
        game_response, reward, done, info = env.step(player_in)
        print(game_response)

        retries += 1
        if info['score'] != prev_score: # If score changes, major step has been made so number of retries resets
            retries_per_score.append(retries)
            retries = 0
        prev_score = info['score']
        total_steps += 1
    
    print('Scored', info['score'], 'out of', env.get_max_score())

    sys.stdout = original_stdout
    return {
        'score': info['score'],
        'max_score': env.get_max_score(),
        'avg_retries': sum(retries_per_score) / len(retries_per_score) if len(retries_per_score) != 0 else 0,
        'avg_generate_time': sum(generate_times) / len(generate_times)
    }

In [12]:
basic_llm()

Playing game. 'q' to quit
Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


[r0] LLM Response: I'm currently standing in an open field west of the white house, with a boarded front door. The field stretches out in both directions, with no visible obstacles or hazards. I see a small mailbox at the far end of the field.

My thought process: Considering my current location and the fact that there is a mailbox, it's likely that my goal is to retrieve something from the mailbox. Given the layout of the scene, I can think of several possible directions to take:

* I could try to open the boarded front door (if it's not locked).
* I could search for another way into or out of the house.
* I could investigate the mailbox more closely.

My command: go north
My command

{'score': 0,
 'max_score': 350,
 'avg_retries': 1.0,
 'avg_generate_time': 31.602583374295914}

In [ ]:
# Basic with memory
system_prompt = (
    'You are a smart video game enthusiast who is skilled at playing old-school text adventure games. Given a description of your environment, explain your thought process and then give a command that is compatible with the game you are playing. Always put the command on its own line at the end of your response with nothing else. It needs to be easy and consistent to read with simple python'
    'At the beginning of your prompt, you will also receive up to 5 of the most recent interactions you\'ve had with the game'
    'This game does not run on an LLM, so it only recognizes a set vocabulary of commands. An example of a command is exactly the string "go west". This means you should only give exactly the command that the game recognizes '
)

In [ ]:
def basic_llm_with_memory(memory_size=5, max_retry=5, out_file=sys.stdout, system_prompt=system_prompt):
    original_stdout = sys.stdout
    sys.stdout = out_file

    make_prompt = lambda x: f'{system_prompt}\n\n{x}'

    print("Playing game. 'q' to quit")

    initial_observation, info = env.reset()
    print(initial_observation)

    game_response = initial_observation

    memory = []

    done = False
    total_steps = 0
    retries = 0
    retries_per_score = []
    generate_times = []
    prev_score = 0
    while not done and retries <= max_retry:

        prompt = make_prompt(f'Game prompt:\n{game_response}')
        combined_memory = "\n".join(memory)
        prompt_with_memory = f'{combined_memory}\n\n{prompt}'
        start = time.time()
        response = ollama.generate(model='llama3.2:1b', prompt=prompt_with_memory).response
        generate_times.append(time.time() - start)
        print(f'[r{retries}] LLM Response: {response}')
        player_in = response.splitlines()[-1].strip()

        memory.append(f'{prompt}\n{response}')
        if len(memory) > memory_size:
            memory.pop(0)

        # Take an action in the environment using the step fuction.
        # The resulting text-observation, reward, and game-over indicator is returned.
        game_response, reward, done, info = env.step(player_in)
        game_response = f'Received command: {player_in}\n{game_response}' # Add text the game received so the LLM can hopefully improve it's formatting
        print(game_response)

        retries += 1
        if info['score'] != prev_score: # If score changes, major step has been made so number of retries resets
            retries_per_score.append(retries)
            retries = 0
        prev_score = info['score']
        total_steps += 1
    
    print('Scored', info['score'], 'out of', env.get_max_score())

    sys.stdout = original_stdout
    return {
        'score': info['score'],
        'max_score': env.get_max_score(),
        'avg_retries': sum(retries_per_score) / len(retries_per_score) if len(retries_per_score) != 0 else 0,
        'avg_generate_time': sum(generate_times) / len(generate_times)
    }

In [9]:
with open('./basic_llm_with_memory.out', 'w') as f:
    print(basic_llm_with_memory(out_file=f, max_retry=30))

{'score': 0, 'max_score': 350, 'avg_retries': 0, 'avg_generate_time': 7.857473757959181}


In [ ]:
# ChatGPT Provided. Is it better?
system_prompt = (
    "You are a player of a classic parser-based interactive fiction game.\n"
    "Respond only with a single imperative command in plain lowercase (e.g., “look”, “go east”, “get key”).\n"
    "Do not write sentences, explanations, strategies, or narratives.\n"
    "Use only standard text-adventure verbs: look, examine, go, take, drop, open, close, use, talk to, attack, inventory, etc.\n"
    "Act rationally based on the game’s last description.\n"
    "If confused, issue “look”.\n"
    "Notice that the memory you receive contains commands you have issued in the past. Don't repeat commands that won't help you move forward.\n"
)

In [ ]:
with open('./basic_llm_with_memory.out', 'w') as f:
    print(basic_llm_with_memory(out_file=f, max_retry=30))

The chatgpt given prompt was ineffective. It tended to repeat commands over and over again, even when we provided the memory of the command failing

In [40]:
# Basic with memory
system_prompt = (
    f'You are a smart video game enthusiast who is skilled at playing old-school text adventure games. You are playing {game}.\n'
    "You don't enjoy playing video games, but you must reach the proper end of the game to get paid.\n"
)

In [ ]:
def memory_analyze_then_command(memory_size=5, max_retry=20, out_file=sys.stdout, model='llama3.2:1b'):
    original_stdout = sys.stdout
    sys.stdout = out_file

    make_prompt = lambda x: f'{system_prompt}\n\n{x}'

    print("Playing game. 'q' to quit")

    initial_observation, info = env.reset()
    print(initial_observation)

    game_response = initial_observation

    memory = []

    done = False
    total_steps = 0
    retries = 0
    retries_per_score = []
    generate_times = []
    prev_score = 0
    while not done and retries <= max_retry:

        combined_memory = "\n".join(memory)
        prompt = make_prompt(f'Analyze the following sequence of game environment descriptions or responses and actions already taken by you, the player. Concisely describe the current state of the game and a potential action to take to move forward.\n\n{combined_memory}\n\nGame text:\n{game_response}')
        start = time.time()
        response = ollama.generate(model=model, prompt=prompt).response
        generate_times.append(time.time() - start)
        print(f'[r{retries}] LLM description: {response}')

        prompt = make_prompt(f'Given your analysis of the game state, issue a rational action to take to progress in the game. Respond only with a single imperative command in plain lowercase. Use only standard text-adventure verbs. IMPORTANT: Your response will be used directly as input to the game. Minimize the number of words you use.\n\nYour analysis:\n{response}')
        start = time.time()
        response = ollama.generate(model=model, prompt=prompt).response
        generate_times.append(time.time() - start)
        print(f'[r{retries}] LLM action: {response}')
        response = response.removeprefix('type').strip() # Give it a shot. Keeps saying type! TODO: Probably remove. Bandaid

        player_in = response.splitlines()[-1].strip()

        memory.append(f'{game_response}\n{response}')
        if len(memory) > memory_size:
            memory.pop(0)

        # Take an action in the environment using the step fuction.
        # The resulting text-observation, reward, and game-over indicator is returned.
        game_response, reward, done, info = env.step(player_in)
        game_response = f'Received command: {player_in}\n{game_response}' # Add text the game received so the LLM can hopefully improve it's formatting
        print(game_response)

        retries += 1
        if info['score'] != prev_score: # If score changes, major step has been made so number of retries resets
            retries_per_score.append(retries)
            retries = 0
        prev_score = info['score']
        total_steps += 1
    
    print('Scored', info['score'], 'out of', env.get_max_score())

    sys.stdout = original_stdout
    return {
        'score': info['score'],
        'max_score': env.get_max_score(),
        'avg_retries': sum(retries_per_score) / len(retries_per_score) if len(retries_per_score) != 0 else 0,
        'avg_generate_time': sum(generate_times) / len(generate_times)
    }

In [5]:
memory_analyze_then_command(max_retry=5, model='gpt-oss:20b')

Playing game. 'q' to quit
Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


[r0] LLM description: **Current state:**  
- You are standing in an open field west of a white house.  
- The front door of the house is boarded up.  
- There is a small mailbox on the ground.

**Suggested next move:**  
Open or examine the mailbox: `open mailbox` or `look mailbox`. This is a classic first step in Zork to retrieve a letter that can give you a clue or key. If you prefer to approach the house, you could also try `go east` or `enter house`, but the mailbox is a more straightforward path to progress.
[r0] LLM action: open mailbox
Received command: open mailbox
Opening the small mailbox reveals a leaflet.


[r1] LLM description: You’re still standing in the open field west

{'score': 0,
 'max_score': 350,
 'avg_retries': 0,
 'avg_generate_time': 60.48966830968857}

gpt-oss did much better than llama. Could be because of the number of parameters. Could also be that gpt-oss has seen zork in its training data (it seems like this is true). Or maybe it's really better at it? I think that's less likely.

In [ ]:
system_prompt = (
    f'You are a smart video game tester who is skilled at playing old-school text adventure games. You are playing {game}\n'
    'Respond only with a single imperative command in plain lowercase from the list of possible actions below.\n'
    'Do not write sentences, explanations, strategies, or narratives.\n'
    'Act rationally based on the game’s last description.\n'
    "Notice that the memory you receive contains commands you have issued in the past. Don't repeat commands that won't help you move forward\n"
)

In [36]:
def memory_and_provided_commands(memory_size=5, max_retry=5, out_file=sys.stdout, system_prompt=system_prompt):
    original_stdout = sys.stdout
    sys.stdout = out_file

    make_prompt = lambda x: f'{system_prompt}\n\n{x}'

    print("Playing game. 'q' to quit")

    initial_observation, info = env.reset()
    print(initial_observation)

    game_response = initial_observation

    memory = []

    done = False
    total_steps = 0
    retries = 0
    retries_per_score = []
    generate_times = []
    prev_score = 0
    while not done and retries <= max_retry:

        combined_memory = "\n".join(memory)
        actions = ', '.join(env.get_valid_actions())
        prompt = make_prompt(f'{combined_memory}\n\nGame text:\n{game_response}\n\nValid actions: {actions}')
        start = time.time()
        response = ollama.generate(model='llama3.2:1b', prompt=prompt).response
        generate_times.append(time.time() - start)
        print(f'[r{retries}] LLM Response: {response}')
        player_in = response.splitlines()[-1].strip()

        memory.append(f'{game_response}\n{response}')
        if len(memory) > memory_size:
            memory.pop(0)

        # Take an action in the environment using the step fuction.
        # The resulting text-observation, reward, and game-over indicator is returned.
        game_response, reward, done, info = env.step(player_in)
        game_response = f'Received command: {player_in}\n{game_response}' # Add text the game received so the LLM can hopefully improve it's formatting
        print(game_response)

        retries += 1
        if info['score'] != prev_score: # If score changes, major step has been made so number of retries resets
            retries_per_score.append(retries)
            retries = 0
        prev_score = info['score']
        total_steps += 1
    
    print('Scored', info['score'], 'out of', env.get_max_score())

    sys.stdout = original_stdout
    return {
        'score': info['score'],
        'max_score': env.get_max_score(),
        'avg_retries': sum(retries_per_score) / len(retries_per_score) if len(retries_per_score) != 0 else 0,
        'avg_generate_time': sum(generate_times) / len(generate_times)
    }

In [38]:
memory_and_provided_commands(max_retry=30)

Playing game. 'q' to quit
Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


[r0] LLM Response: open mailbox
Received command: open mailbox
Opening the small mailbox reveals a leaflet.


[r1] LLM Response: take leaflet
Received command: take leaflet
Taken.


[r2] LLM Response: put down leaflet
Received command: put down leaflet
Dropped.


[r3] LLM Response: close mailbox
Received command: close mailbox
Closed.


[r4] LLM Response: open door
Received command: open door
The door cannot be opened.


[r5] LLM Response: north
Received command: north
North of House
You are facing the north side of a white house. There is no door here, and all the windows are boarded up. To the north a narrow path winds through the trees.


[r6] LLM Response: open door
Received comma

{'score': 0,
 'max_score': 350,
 'avg_retries': 0,
 'avg_generate_time': 1.5271822021853538}

This certainly improved it's commands, but not for actually completing the game. Maybe having an analysis step could help. It does not have a sense of curiosity

In [7]:
def memory_analyze_provided_commands(memory_size=5, max_retry=5, out_file=sys.stdout, model='llama3.2:1b'):
    original_stdout = sys.stdout
    sys.stdout = out_file

    make_prompt = lambda x: f'{system_prompt}\n\n{x}'

    print("Playing game. 'q' to quit")

    initial_observation, info = env.reset()
    print(initial_observation)

    game_response = initial_observation

    memory = []

    done = False
    total_steps = 0
    retries = 0
    retries_per_score = []
    generate_times = []
    prev_score = 0
    while not done and retries <= max_retry:

        combined_memory = "\n".join(memory)
        prompt = make_prompt(f'Analyze the following sequence of game environment descriptions or responses and actions already taken by you, the player. Concisely describe the current state of the game and a potential action to take to move forward.\n\n{combined_memory}\n\nGame text:\n{game_response}')
        start = time.time()
        response = ollama.generate(model=model, prompt=prompt).response
        generate_times.append(time.time() - start)
        print(f'[r{retries}] LLM description: {response}')

        actions = ', '.join(env.get_valid_actions())
        prompt = make_prompt(f'Given your analysis of the game state, issue a rational action to take to progress in the game. Respond only with a single imperative command in plain lowercase. Use only standard text-adventure verbs. IMPORTANT: Your response will be used directly as input to the game. Minimize the number of words you use.\n\nYour analysis:\n{response}\n\nValid actions: {actions}')
        start = time.time()
        response = ollama.generate(model=model, prompt=prompt).response
        generate_times.append(time.time() - start)
        print(f'[r{retries}] LLM action: {response}')
        response = response.removeprefix('type').strip() # Give it a shot. Keeps saying type! TODO: Probably remove. Bandaid

        player_in = response.splitlines()[-1].strip()

        memory.append(f'{game_response}\n{response}')
        if len(memory) > memory_size:
            memory.pop(0)

        # Take an action in the environment using the step fuction.
        # The resulting text-observation, reward, and game-over indicator is returned.
        game_response, reward, done, info = env.step(player_in)
        game_response = f'Received command: {player_in}\n{game_response}' # Add text the game received so the LLM can hopefully improve it's formatting
        print(game_response)

        retries += 1
        if info['score'] != prev_score: # If score changes, major step has been made so number of retries resets
            retries_per_score.append(retries)
            retries = 0
        prev_score = info['score']
        total_steps += 1
    
    print('Scored', info['score'], 'out of', env.get_max_score())

    sys.stdout = original_stdout
    return {
        'score': info['score'],
        'max_score': env.get_max_score(),
        'avg_retries': sum(retries_per_score) / len(retries_per_score) if len(retries_per_score) != 0 else 0,
        'avg_generate_time': sum(generate_times) / len(generate_times)
    }

In [9]:
memory_analyze_provided_commands(max_retry=30)

Playing game. 'q' to quit
Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


[r0] LLM description: Based on the game environment descriptions and responses, I can summarize the current state of the game as follows:

The player has just entered a new area, a field west of a white house. The player's current location is indicated by the text "West of House". There are two objects in this area: an open field with no specific description or features, and a small mailbox.

Given my previous actions (not specified), I can infer that:

* The player has entered the field.
* The player may have exited the house at some point to access the field.
* The player is likely familiar with the basic layout of this area.

A potential action for me to take to move forward could 

{'score': 0,
 'max_score': 350,
 'avg_retries': 0,
 'avg_generate_time': 6.211477264281242}

In [41]:

def memory_analyze_provided_commands_chat(memory_size=5, max_retry=20, out_file=sys.stdout, model='llama3.2:1b'):
    original_stdout = sys.stdout
    sys.stdout = out_file

    make_prompt = lambda x: f'{system_prompt}\n\n{x}'

    print("Playing game. 'q' to quit")

    initial_observation, info = env.reset()
    print(initial_observation)

    game_response = initial_observation

    memory = [
        ollama.Message(role='system', content=system_prompt)
    ]
    analysis_prompt = 'Concisely describe the current state of the game and a potential action to take to move forward.'
    memory.append(ollama.Message(role='system', content=analysis_prompt))

    done = False
    total_steps = 0
    retries = 0
    retries_per_score = []
    generate_times = []
    prev_score = 0
    while not done and retries <= max_retry:

        actions_list = env.get_valid_actions()
        random.shuffle(actions_list)
        actions = ', '.join(actions_list)

        memory.append(ollama.Message(role='user', content=f'{game_response}\n\nValid game actions: {actions}'))
        start = time.time()
        response = ollama.chat(model=model, messages=memory)
        memory.append(response.message)
        response = response.message.content
        generate_times.append(time.time() - start)
        print(f'[r{retries}] LLM description: {response}')

        prompt = make_prompt(f'Given your analysis of the game state, issue a rational action to take to progress in the game. Respond only with a single imperative command in plain lowercase. Use only standard text-adventure verbs. IMPORTANT: Your response will be used directly as input to the game. Minimize the number of words you use.\n\nYour analysis:\n{response}\n\n Only use one of these valid actions: {actions}\n\n')
        print('[action prompt]', prompt)
        start = time.time()
        response = ollama.generate(model=model, prompt=prompt).response
        generate_times.append(time.time() - start)
        print(f'[r{retries}] LLM action: {response}')
        response = response.removeprefix('type').strip() # Give it a shot. Keeps saying type! TODO: Probably remove. Bandaid

        player_in = response.splitlines()[-1].strip()

        # Take an action in the environment using the step fuction.
        # The resulting text-observation, reward, and game-over indicator is returned.
        game_response, reward, done, info = env.step(player_in)
        game_response = f'Received command: {player_in}\n{game_response}' # Add text the game received so the LLM can hopefully improve it's formatting
        print(game_response)

        retries += 1
        if info['score'] != prev_score: # If score changes, major step has been made so number of retries resets
            retries_per_score.append(retries)
            retries = 0
        prev_score = info['score']
        total_steps += 1
    
    print('Scored', info['score'], 'out of', env.get_max_score())
    print('Memory at end:')
    print(json.dumps(memory, indent=4, default=str))


    sys.stdout = original_stdout
    return {
        'score': info['score'],
        'max_score': env.get_max_score(),
        'avg_retries': sum(retries_per_score) / len(retries_per_score) if len(retries_per_score) != 0 else 0,
        'avg_generate_time': sum(generate_times) / len(generate_times)
    }

In [ ]:
memory_analyze_provided_commands_chat()

Playing game. 'q' to quit
Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


[r0] LLM description: I'm currently standing in the middle of this open field, surrounded by tall grass and wildflowers. The only landmarks are a few scattered rocks and a tall wooden fence on the east side of the field. In front of me is a small mailbox with a rusty lock.

To move forward, I can:

* Open the mailbox to investigate its contents
* Head north towards the boarded front door of the white house

I'm not really sure what's going on here, but I don't see any obvious way out...
[action prompt] You are a smart video game enthusiast who is skilled at playing old-school text adventure games. You are playing zork1.z5.
You don't enjoy playing video games, but you must reach the pr

Process SpawnPoolWorker-42:
Process SpawnPoolWorker-41:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 365, in ge

KeyboardInterrupt: 

In [ ]:
system_prompt = (
    f'Think step by step. You are playing {game}, an interactive fiction game. You must analyze the scenario the game presents to you and choose an action that will make progress. Your goal is to finish the game\n'
    'Use the tools provided to you to take actions, view possible actions, and view the game walkthrough if necessary'
)

def agent(max_retry=20, out_file=sys.stdout, model='qwen3'):
    original_stdout = sys.stdout
    sys.stdout = out_file

    print("Playing game. 'q' to quit")

    initial_observation, info = env.reset()
    print(initial_observation)

    game_response = initial_observation

    memory = [
        ollama.Message(role='system', content=system_prompt)
    ]

    done = False

    def do_game_action(action: str) -> str:
        """Perform an action in the active text adventure game and see the result"""
        """
        Args:
          action: game action string

        Returns:
          The game's response after performing the action
        """
        game_response, reward, done, info = env.step(action)
        return game_response
    
    def view_possible_actions() -> str:
        """View a list of the actions that can be performed in the game's current state"""
        """
        Returns:
          String containg actions separated by commas
        """
        return ', '.join(env.get_valid_actions())
    
    def view_walkthrough():
        """View the full game walkthrough as a list of actions"""
        """
        Returns:
          String containing actions separated by newlines
        """
        return env.get_walkthrough()
    
    available_functions = {
        'do_game_action': do_game_action,
        'view_possible_actions': view_possible_actions,
        'view_walkthrough': view_walkthrough
    }

    total_steps = 0
    retries = 0
    retries_per_score = []
    generate_times = []
    prev_score = 0
    while not done and retries <= max_retry:

        memory.append(ollama.Message(role='user', content=f'{game_response}'))

        while True: # Keep looping until it finishes tool calls (which is kind of never)
            start = time.time()
            response = ollama.chat(model=model, messages=memory, think=True, tools=[do_game_action, view_possible_actions, view_walkthrough])
            generate_times.append(time.time() - start)
            memory.append(response.message)

            print("Thinking: ", response.message.thinking)
            print("Content: ", response.message.content)

            if response.message.tool_calls:
                for tc in response.message.tool_calls:
                    if tc.function.name in available_functions:
                        print(f"Calling {tc.function.name} with arguments {tc.function.arguments}")
                        result = available_functions[tc.function.name](**tc.function.arguments)
                        print(f"Result: {result}")
                        # add the tool result to the messages
                        memory.append({'role': 'tool', 'tool_name': tc.function.name, 'content': str(result)})
            else:
                break
        
        # print(f'[r{retries}] LLM description: {response}')
        # player_in = response.splitlines()[-1].strip()

        # Take an action in the environment using the step fuction.
        # The resulting text-observation, reward, and game-over indicator is returned.
        # game_response, reward, done, info = env.step(player_in)
        # game_response = f'Received command: {player_in}\n{game_response}' # Add text the game received so the LLM can hopefully improve it's formatting
        # print(game_response)

        retries += 1
        if info['score'] != prev_score: # If score changes, major step has been made so number of retries resets
            retries_per_score.append(retries)
            retries = 0
        prev_score = info['score']
        total_steps += 1
    
    print('Scored', info['score'], 'out of', env.get_max_score())
    print('Memory at end:')
    print(json.dumps(memory, indent=4, default=str))


    sys.stdout = original_stdout
    return {
        'score': info['score'],
        'max_score': env.get_max_score(),
        'avg_retries': sum(retries_per_score) / len(retries_per_score) if len(retries_per_score) != 0 else 0,
        'avg_generate_time': sum(generate_times) / len(generate_times)
    }

In [13]:
agent()

Playing game. 'q' to quit
Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


Thinking:  Okay, let's see. I'm playing Zork I, and I'm at the starting point: West of House. The description says there's a small mailbox here. The goal is to finish the game, so I need to figure out what to do next.

First, I should check what actions are possible here. The user mentioned using the view_possible_actions function. Let me call that to see the available options. Common actions in such games usually involve looking around, opening doors, checking containers, or interacting with objects. Since there's a mailbox, maybe I can open it or check its contents. Also, the front door is boarded, so perhaps I need to break it or find a key. But without knowing the exact commands, 

Process SpawnPoolWorker-1:
Process SpawnPoolWorker-7:
Process SpawnPoolWorker-6:
Process SpawnPoolWorker-4:
Process SpawnPoolWorker-9:
Process SpawnPoolWorker-5:
Process SpawnPoolWorker-8:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-2:
Process SpawnPoolWorker-10:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 365, i

KeyboardInterrupt: 